In [ ]:
# ask pdf

In [3]:
# import model 
from langchain_community.llms import LlamaCpp

llm_pdf=LlamaCpp(
    model_path=r"C:\Users\ASUS\Downloads\Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    seed=42,
    verbose=True,
    n_ctx=2048
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from C:\Users\ASUS\Downloads\Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.he

In [5]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#Getting the pdf
from PyPDF2 import PdfReader
pdf=r"C:\Users\ASUS\Downloads\508_Ds\Chapter 1 - Array List and Linked List.pdf"
pdf_reader=PdfReader(pdf)

#get text in each pages
text=""
for page in pdf_reader.pages:
    text+=page.extract_text()

In [17]:
#split the chunks text (make it smaller)
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200, #Overlap helps the model not “forget” context when processing chunks separately.
    length_function=len
)
chunks=text_splitter.split_text(text)

In [11]:
pip install -q sentence_transformers==2.3.0

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.1.2 requires sentence-transformers>=2.6.0, but you have sentence-transformers 2.3.0 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11040\3879360024.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
pip install faiss-cpu

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------- ----------------------------- 5.0/18.2 MB 30.6 MB/s eta 0:00:01
   ---------------------------- ----------- 12.8/18.2 MB 34.9 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 32.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
#embedding the text chunks, creating Vector DAtabase
from langchain.vectorstores import FAISS
Vector_db=FAISS.from_texts(chunks,embedding_function)

In [42]:
#RAG PROMPT
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

template="""<|user|>
Relevant Information:
{context}

Provide a concise answer the following question using the relevant and only information provided above.
Use only sentences from the context.
 

{question} <|end|>
<|assistance|>

"""

prompt=PromptTemplate(
    template=template,
    input_variables=["context","question"]
)

rag = RetrievalQA.from_chain_type(
    llm=llm_pdf,
    chain_type='stuff',
    retriever=Vector_db.as_retriever(),
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=True,
    return_source_documents=True
)

In [44]:
rag.invoke("Define Circular Linked List and its advantage and disadvantage")



> Entering new RetrievalQA chain...


Llama.generate: 295 prefix-match hit, remaining 993 prompt tokens to eval
llama_perf_context_print:        load time =   53212.84 ms
llama_perf_context_print: prompt eval time =   38633.66 ms /   993 tokens (   38.91 ms per token,    25.70 tokens per second)
llama_perf_context_print:        eval time =   23260.83 ms /   105 runs   (  221.53 ms per token,     4.51 tokens per second)
llama_perf_context_print:       total time =   61950.98 ms /  1098 tokens
llama_perf_context_print:    graphs reused =        194



> Finished chain.


{'query': 'Define Circular Linked List and its advantage and disadvantage',
 'result': 'A circular linked list is a variation of the linked list in which every node points to the next node, and the last node points back to the first node. This creates a loop that allows traversal in either direction from any point on the list. The advantage of a circular linked list includes being able to visit all elements from any starting point without needing to reach an end or beginning, as it forms a continuous cycle; however, its disadvantage is the potential for infinite loops if not managed correctly after removing nodes.',
 'source_documents': [Document(id='ce05fa04-4ee0-4e53-9647-fd824f9581d4', metadata={}, page_content='▪Can insert a node only after a referenced node\n▪Can remove a node only if we have a reference to \nits predecessor node\n▪Can traverse the list only in the forward direction\n▪Above limitations removed by adding a \nreference in each node to the previous node \n(double -li

In [48]:
rag.invoke("Define mechanical engineer")



> Entering new RetrievalQA chain...


Llama.generate: 10 prefix-match hit, remaining 1182 prompt tokens to eval
llama_perf_context_print:        load time =   53212.84 ms
llama_perf_context_print: prompt eval time =   47364.07 ms /  1182 tokens (   40.07 ms per token,    24.96 tokens per second)
llama_perf_context_print:        eval time =   28170.88 ms /   127 runs   (  221.82 ms per token,     4.51 tokens per second)
llama_perf_context_print:       total time =   75608.68 ms /  1309 tokens
llama_perf_context_print:    graphs reused =        231



> Finished chain.


{'query': 'Define mechanical engineer',
 'result': 'Unfortunately, the provided information does not specifically define what a mechanical engineer is. However, based on general knowledge within the context of engineering and data structures:\n\nA mechanical engineer may be involved in designing mechanical systems and components that could include elements such as gears or linkages (similar to linked lists), although this topic\'s focus was more about programming concepts rather than specific job roles like a mechanical engineer. To define a mechanical engineer, we would typically say:\n\n"A mechanical engineer is a professional who applies principles of physics and materials science for the design, analysis, manufacturing, and maintenance of mechanical systems."',
 'source_documents': [Document(id='a5b3ce90-7ab2-427f-a4ca-9f77f5c7bf3f', metadata={}, page_content='▪Delete the last item from the list\n21LINKED LISTS: SOME PROPERTIES▪Operations require traversal of the list.\n▪Given a re